# Convert annotations to MS Coco format with more meta fields

Convert datasets: validation and test
```
validation -> train
test -> val
```
We added into annotations:
- 'IsOccluded'
- 'IsTruncated'
- 'IsDepiction'
- 'IsInside'

Apply prefiltering: 
- images with any dimension larger 2000 pixels are ignored
- do not write annotations without bboxes

In [1]:
from pathlib import Path

import numpy as np
import pandas as pd

import json
from PIL import Image

In [2]:
VALIDATION_IMAGES_PATH = Path(".").resolve().parent / "input" / "validation"
VALIDATION_ANNOTATIONS_CSV_PATH = Path(".").resolve().parent / "input" / "validation-annotations-bbox.csv"
VALIDATION_CONFIDENCE_CSV_PATH = Path(".").resolve().parent / "input" / "validation-annotations-human-imagelabels-boxable.csv"
VALIDATION_IMGINFO_CSV_PATH = Path(".").resolve().parent / "input" / "validation-images-with-rotation.csv"
LABELS_DESCRIPTION_CSV_PATH = Path(".").resolve().parent / "input" / "class-descriptions-boxable.csv"

In [3]:
TEST_IMAGES_PATH = Path(".").resolve().parent / "input" / "test"
TEST_ANNOTATIONS_CSV_PATH = Path(".").resolve().parent / "input" / "test-annotations-bbox.csv"
TEST_CONFIDENCE_CSV_PATH = Path(".").resolve().parent / "input" / "test-annotations-human-imagelabels-boxable.csv"

In [4]:
# mode = "val"
# images_path = VALIDATION_IMAGES_PATH
# annotations_path = VALIDATION_ANNOTATIONS_CSV_PATH
# output_mode = mode

mode = "test"
images_path = TEST_IMAGES_PATH
annotations_path = TEST_ANNOTATIONS_CSV_PATH
output_mode = mode

In [5]:
annotations = pd.read_csv(annotations_path, index_col="ImageID")
labels_description = pd.read_csv(LABELS_DESCRIPTION_CSV_PATH, header=None)
annotations['LabelName'] = annotations['LabelName'].map(labels_description.set_index(0)[1])

xyxy_cols = ['XMin', 'YMin', 'XMax', 'YMax']
meta_cols = ['IsOccluded', 'IsTruncated', 'IsGroupOf', 'IsDepiction', 'IsInside']


def get_bboxes_labels_meta(canvas_size, image_id):
    bboxes = annotations.loc[image_id, xyxy_cols].values
    labels = annotations.loc[image_id, 'LabelName']
    meta = annotations.loc[image_id, meta_cols].values
    
    if bboxes.ndim == 1:
        bboxes = bboxes[None, :]
        meta = meta[None, :]
    
    if isinstance(labels, str):        
        labels = np.array([labels, ])
    bboxes[:, 0] *= canvas_size[0]
    bboxes[:, 2] *= canvas_size[0]    
    bboxes[:, 1] *= canvas_size[1]
    bboxes[:, 3] *= canvas_size[1]
    return bboxes, labels, meta


def compute_area(bbox):
    w = bbox[2] - bbox[0] 
    h = bbox[3] - bbox[1]    
    return w * h


In [6]:
labels = labels_description[1].values.tolist()
image_ids = annotations.index.unique()

In [7]:
coco_categories = []
for i, label in enumerate(labels):
    coco_categories.append({
        'id': i,
        'name': label,
        'supercategory': label
    })

In [8]:
categories = {}
for d in coco_categories:
    categories[d['name']] = d['id']

In [9]:
annotations.head()

,Source,LabelName,Confidence,XMin,XMax,YMin,YMax,IsOccluded,IsTruncated,IsGroupOf,IsDepiction,IsInside
ImageID,,,,,,,,,,,,
000026e7ee790996,freeform,Tree,1,0.071905,0.145346,0.206591,0.391306,0,1,1,0,0
000026e7ee790996,freeform,Tree,1,0.439756,0.572466,0.264153,0.435122,0,1,1,0,0
000026e7ee790996,freeform,Tree,1,0.668455,1.000000,0.000000,0.552825,0,1,1,0,0
000062a39995e348,freeform,Bird,1,0.205719,0.849912,0.154144,1.000000,0,0,0,0,0
000062a39995e348,freeform,Plant,1,0.137133,0.377634,0.000000,0.884185,1,1,0,0,0


In [10]:
coco_images = []
coco_annotations = []


for image_id in image_ids:    
    img = Image.open(images_path / "{}.jpg".format(image_id))
    
    if max(img.size) > 2000 or min(img.size) < 100:
        continue
    
    image_info = {
            "id": image_id,
            "file_name": "{}.jpg".format(image_id),
            "width": img.size[0],
            "height": img.size[1],
    }    
    bboxes, labels, meta = get_bboxes_labels_meta(img.size, image_id)
    
    if len(bboxes) == 0:
        print("No bboxes for image_id '{}'".format(image_id))
        continue

    coco_images.append(image_info)    
    for i, (bbox, label, m) in enumerate(zip(bboxes, labels, meta)):
        m = [int(v) for v in m]
        annotation_id = image_id + "_{}".format(i)
        annotation_info = {
            "id": annotation_id,
            "image_id": image_id,
            "category_id": categories[label],
            "IsOccluded": m[0],
            "IsTruncated": m[1],
            "iscrowd": m[2],
            "IsDepiction": m[3],
            "IsInside": m[4],            
            "area": int(compute_area(bbox)),
            "bbox": [int(v) for v in bbox.tolist()],
            "segmentation": [],
        } 
        coco_annotations.append(annotation_info)    

/usr/local/lib/python3.5/dist-packages/PIL/Image.py:2514: DecompressionBombWarning: Image size (102539736 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning)


In [11]:
output_coco_annotations = {
    "categories": coco_categories,
    "images": coco_images,
    "annotations": coco_annotations
}

In [12]:
output_folder = Path(".").resolve().parent / "input" / "as_mscoco" / "annotations" 
if not output_folder.exists():
    output_folder.mkdir(parents=True)

In [13]:
with (output_folder / "{}.json".format(output_mode)).open('w') as h:
    json.dump(output_coco_annotations, h)

In [93]:
if not output_images_folder.exists():
    output_images_folder = Path(".").resolve().parent / "input" / "as_mscoco" / output_mode
    output_images_folder.symlink_to(images_path, target_is_directory=True)

Test with pycocotools

In [71]:
from pycocotools import coco

In [72]:
coco = coco.COCO((output_folder / "val.json").as_posix())

loading annotations into memory...
Done (t=1.39s)
creating index...
index created!


In [73]:
anns = coco.getAnnIds()

In [76]:
len(anns), anns[:2]

(204621, ['0001eeaf4aed83f9_0', '000595fe6fee6369_0'])

In [4]:
import multiprocessing as mp

import random
import numpy as np

def task():
    print("-- Task --")
    print(random.random(), np.random.rand())
    print(random.random(), np.random.rand())
    print("-- End Task --")
    
random.seed(12345)
np.random.seed(12345)

workers = [mp.Process(target=task, args=()) for i in range(4)]

print("Run")
for w in workers:
    w.start()

Run
-- Task --
0.6062518296570525 0.9296160928171479
0.7142844140096419 0.3163755545817859
-- End Task --
-- Task --
0.6959326394013083 0.9296160928171479
0.31228532647566887 0.3163755545817859
-- End Task --
-- Task --
0.4936887984487045 0.9296160928171479
0.7509582156538067 0.3163755545817859
-- Task --
-- End Task --
0.6168844031764428 0.9296160928171479
0.5782864572237567 0.3163755545817859
-- End Task --
